In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go 

In [ ]:
file = "../final_sdf.csv"

In [ ]:
df = pd.read_csv(file)
df.head()

,x,y,z,value
0,-450.0,-450.0,-250.000,10.9926
1,-450.0,-450.0,-218.966,10.5653
2,-450.0,-450.0,-187.931,10.4878
3,-450.0,-450.0,-156.897,10.4877
4,-450.0,-450.0,-125.862,10.4877


In [ ]:
fig = go.Figure(data=go.Isosurface(
    x=df.x,
    y=df.y,
    z=df.z,
    value=df.value,
    surface_count=1,
    # slices= {
    #     'x': {'show': True, 'locations': [df.x.max()/2]},
    #     'y': {'show': True, 'locations': [df.y.max()/2]},
    #     'z': {'show': True, 'locations': [df.z.max()/2]}
    # },
    caps=dict(x_show=False, y_show=False, z_show=False),
    colorscale='Viridis',
    # opacity=0.6
))

# Add axis labels and title
fig.update_layout(
    title='3D Volumetric Slice Visualization',
    scene=dict(
        xaxis_title='X Axis',
        yaxis_title='Y Axis',
        zaxis_title='Z Axis'
    )
)
fig.show()

derived the backward Euler implementation for the level set equation by starting with the standard level set equation and applying implicit time discretization. Here's the mathematical derivation:

### Level Set Equation
The level set equation is a partial differential equation that describes the evolution of an interface:

$$\frac{\partial \phi}{\partial t} + \vec{V} \cdot \nabla \phi = 0$$

Where:

- $\phi$ is the level set function
- $\vec{V} = (V_x, V_y, V_z)$ is the velocity field
- $\nabla \phi$ is the gradient of $\phi$
### Backward Euler Discretization
The backward Euler method is an implicit time integration scheme. For the level set equation, it takes the form:

$$\frac{\phi^{n+1} - \phi^n}{\Delta t} + \vec{V} \cdot \nabla \phi^{n+1} = 0$$

Rearranging to solve for $\phi^{n+1}$:

$$\phi^{n+1} - \Delta t \vec{V} \cdot \nabla \phi^{n+1} = \phi^n$$

This can be written as a linear system:

$$(I - \Delta t L)\phi^{n+1} = \phi^n$$

Where $L$ is the spatial discretization operator for $\vec{V} \cdot \nabla \phi$.

### Numerical Implementation
In the code, I implemented this as follows:

1. Matrix Construction : I built a sparse matrix representing $(I - \Delta t L)$ using triplets:
   
   - The diagonal entries contain $1 + \epsilon$ (where $\epsilon$ is a small regularization term)
   - The off-diagonal entries represent the spatial discretization of $\vec{V} \cdot \nabla \phi$
2. Upwind Scheme : For stability, I used an upwind scheme for the spatial derivatives:
   
   - When velocity is positive, I used backward differences
   - When velocity is negative, I used forward differences
3. Boundary Handling : At boundaries, I used the identity equation $\phi^{n+1} = \phi^n$ to prevent instability
4. Solver : I used the BiCGSTAB solver which is more robust for non-symmetric matrices that arise from the upwind discretization
The key insight in the implementation was recognizing that the matrix structure depends on the flow direction at each point. For example, if flow is from left to right (positive x-velocity), we need to use information from the left neighbor, creating connections in the matrix that reflect this dependency.